In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'D:\\Projects\\ML_Project_end'

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [7]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [8]:
from dataclasses import dataclass
from pathlib import Path

'''Defining data type of the function'''
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir : Path
    all_schema : dict

In [9]:
from src.MLProject.constants import *
from src.MLProject.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_roots])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_dir = config.unzip_data_dir,
            STATUS_FILE = config.STATUS_FILE,
            all_schema = schema,
        )
        
        return data_validation_config
        

In [11]:
import os
from src.MLProject import  logger

In [12]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try:
            validation_status = None
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            # all_types = dict(data.info)
            
            all_schema = self.config.all_schema.keys()
            # all_schema_values = self.config.all_schema.values()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status {validation_status}")
            return validation_status
        except Exception as e:
            raise e
        

In [13]:
## Pipeline

try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-05-08 19:16:53,559: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-08 19:16:53,561: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-08 19:16:53,563: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-08 19:16:53,564: INFO: common: created directory at: artifacts]
[2024-05-08 19:16:53,565: INFO: common: created directory at: artifacts/data_validation]
